In [186]:
%load_ext autoreload
%autoreload 2

import numpy as np
from investment_planner import calculateTransitionPropabilitiesForAllPorfolios, generateGrid, calculateTransitionPropabilities, get_portfolios_strategies, calculateValuesForLastPeriod, get_goals_strategies, __get_value_index
from scipy.stats import norm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [172]:

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
W0 = np.array([90,95,100,105])
W1 = np.array([100,103,105,110])
infusions = 2
cost = 3
h = 1

values = [0,0,100,100]

#propabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios, W0, W1, infusions, cost, h)
propabilities = np.array([[[0.451, 0.334, 0.197, 0.018],
                                    [0.271, 0.27,  0.258, 0.2  ],
                                    [0.255, 0.256, 0.253, 0.237]],

                                    [[0.14,  0.307, 0.366, 0.187],
                                    [0.221, 0.254, 0.266, 0.259],
                                    [0.24,  0.251, 0.255, 0.254]],

                                    [[0.017, 0.105, 0.244, 0.634],
                                    [0.177, 0.234, 0.268, 0.322],
                                    [0.227, 0.246, 0.256, 0.271]],

                                    [[0.001, 0.017, 0.075, 0.908],
                                    [0.14,  0.211, 0.262, 0.387],
                                    [0.215, 0.241, 0.257, 0.288]]])
print(np.round(propabilities, 3))
values = propabilities*values
np.round(np.amax(values.sum(2),1),3)
#values.sum(2)


[[[0.451 0.334 0.197 0.018]
  [0.271 0.27  0.258 0.2  ]
  [0.255 0.256 0.253 0.237]]

 [[0.14  0.307 0.366 0.187]
  [0.221 0.254 0.266 0.259]
  [0.24  0.251 0.255 0.254]]

 [[0.017 0.105 0.244 0.634]
  [0.177 0.234 0.268 0.322]
  [0.227 0.246 0.256 0.271]]

 [[0.001 0.017 0.075 0.908]
  [0.14  0.211 0.262 0.387]
  [0.215 0.241 0.257 0.288]]]


array([49. , 55.3, 87.8, 98.3])

In [173]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057], [0.0886, 0.1954]])
W0 = 100
T = 12
#infusions = [0,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000]
infusions = np.zeros(13)
goals = [0,0,0,0,0,0,0,100,0,0,0,0,150]
iMax = 281

grid = generateGrid(W0,T,iMax, infusions, goals, portfolios[0,0], portfolios[0,1], portfolios[2,0], portfolios[2,1])
grid = grid.T

In [174]:
W0 = 100
meanMax = 0.0886
stdMin = 0.0374
stdMax = 0.1954
t =11

W0*np.exp((meanMax-(stdMin**2/2))*t + 3*stdMax*np.sqrt(t))

1837.723

In [175]:
k = np.array([[105,100],[110,200]])
wealthInT = np.array([90,95,100,105,110])
Wk = np.tile(wealthInT, (2,1))
ki = np.tile(k, (2, 5,1))
#np.where(Wk >= k[:,0], k[:,1], 0)
ki

array([[[105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200]],

       [[105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200]]])

In [176]:
WT1 = np.array([90,95,100,105,110])
WT1P = WT1 * [0.2,0.35, 0.2 ,0.15,0.1]
WT1P.reshape((5,1))

array([[18.  ],
       [33.25],
       [20.  ],
       [15.75],
       [11.  ]])

In [177]:
arr = np.array([[1,2,3,4],[1,2,3]])
arr2 = np.array([1,2])
result = arr*arr2
result

C:\Users\matsz\AppData\Local\Temp\ipykernel_28528\4293703242.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array([[1,2,3,4],[1,2,3]])


array([list([1, 2, 3, 4]), list([1, 2, 3, 1, 2, 3])], dtype=object)

In [178]:
def get_ef_port_analytically(rets, covm, targ):
    """Solve for the efficient frontier weights for a given expected return
    vector `rets`, covariance matrix `covm`, and expected portfolio return
    `targ`.
    """
    N = rets.shape[0]
    u = np.array([targ, 1])[:, None]
    U = np.vstack([rets, np.ones_like(rets)]).T

    covm_inv = np.linalg.solve(covm, np.eye(N))
    M        = U.T @ covm_inv @ U
    M_inv    = np.linalg.solve(M, np.eye(2))
    weights  = covm_inv @ U @ M_inv @ u

    return weights


In [179]:
cov_tab = np.array([[0.0002, -0.00],
                    [-0.00, 0.0013]])
means = np.array([0.0307, 0.1094])

mu = 0.0744
std = 0.0712

w = get_ef_port_analytically(means,cov_tab,mu)

w

array([[0.44472681],
       [0.55527319]])

In [180]:
arr = np.empty((3,3))

arr[:] = np.nan
arr




array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

In [181]:
mean = 0.0886
std = 0.1954
W0 = 105
W1 = np.array([105,110,115])
h= 1
np.round(norm.pdf((np.log(W1/(W0))-(mean-0.5*std**2)*h)/(std*np.sqrt(h))),3)

#100 [0.39670583, 0.39547966, 0.37397359]
#105 [0.37448256, 0.39619069, 0.39654302]


array([0.374, 0.396, 0.397])

In [182]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
W0 = np.array([90,95,100,105])
W1 = np.array([100,103,105,110])
infusions = 2
cost = 50
h = 1

values = [0,0,100,100]

propabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios, W0, W1, infusions, cost, h)
''' propabilities = np.array([[[0.451, 0.334, 0.197, 0.018],
                                    [0.271, 0.27,  0.258, 0.2  ],
                                    [0.255, 0.256, 0.253, 0.237]],

                                    [[0.14,  0.307, 0.366, 0.187],
                                    [0.221, 0.254, 0.266, 0.259],
                                    [0.24,  0.251, 0.255, 0.254]],

                                    [[0.017, 0.105, 0.244, 0.634],
                                    [0.177, 0.234, 0.268, 0.322],
                                    [0.227, 0.246, 0.256, 0.271]],

                                    [[0.001, 0.017, 0.075, 0.908],
                                    [0.14,  0.211, 0.262, 0.387],
                                    [0.215, 0.241, 0.257, 0.288]]]) '''
print(np.round(propabilities, 3))
values = propabilities*values
np.round(np.amax(values.sum(2),1),3)

[[[1.    0.    0.    0.   ]
  [0.888 0.091 0.02  0.   ]
  [0.499 0.266 0.174 0.06 ]]

 [[1.    0.    0.    0.   ]
  [0.848 0.119 0.032 0.001]
  [0.466 0.271 0.188 0.075]]

 [[1.    0.    0.    0.   ]
  [0.8   0.149 0.048 0.003]
  [0.436 0.274 0.2   0.09 ]]

 [[1.    0.    0.    0.   ]
  [0.747 0.18  0.068 0.006]
  [0.408 0.275 0.211 0.106]]]


array([23.493, 26.289, 29.014, 31.661])

In [183]:
def __get_value_index(WT, wealth_value):
    difference = np.absolute(WT - wealth_value)
    index = np.argmin(difference)
    return index



In [452]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
WT = np.array([49,50,60,80,100,105])
WT1 = [90,95,100,103,105,110]
VT1 = [0,0,0,100,100,100]
c = 50
#goals = [[50, 80], [30, 50],]
VTc0 = np.array([29.014, 29.55, 77.286, 95.997])

probabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios,WT,WT1,3,0)

portfolios_strategies, VT, chosen_propabilities = get_portfolios_strategies(VT1,probabilities)
print(portfolios_strategies)
print(VT)
print(np.round(chosen_propabilities,3))
chosen_propabilities.shape

[2 2 2 2 0 0]
[14.79437951 15.58145938 24.11574646 41.94244819 95.45553882 99.66923255]
[[[0.496 0.242 0.114 0.072 0.    0.   ]]

 [[0.484 0.243 0.118 0.075 0.    0.   ]]

 [[0.373 0.239 0.146 0.107 0.    0.   ]]

 [[0.208 0.197 0.175 0.158 0.    0.   ]]

 [[0.113 0.146 0.172 0.184 0.326 0.446]]

 [[0.097 0.134 0.169 0.187 0.126 0.834]]]


(6, 1, 6)

In [454]:
%precision 3


goals = np.array([[40,10],[49, 15],[60, 30]])
#goals = np.array([[0,0]])
infusion = 5

Wt = WT

goal_strategies, portfolio_strategies, values, probabilities = get_goals_strategies(goals, infusion, WT,WT1, VT1, portfolios)


print(goal_strategies)
print(portfolio_strategies)
print(np.round(values,3))
print(np.round(probabilities,3))


[0 0 0 0 0 0]
[2 2 2 2 0 0]
[16.384 17.2   25.916 43.611 98.276 99.897]
[[[0.472 0.243 0.121 0.078 0.    0.   ]
  [0.461 0.244 0.124 0.081 0.    0.   ]
  [0.353 0.237 0.151 0.113 0.    0.   ]
  [0.196 0.192 0.176 0.161 0.    0.   ]
  [0.107 0.141 0.171 0.185 0.244 0.633]
  [0.092 0.129 0.167 0.188 0.075 0.908]]

 [[0.211 0.187 0.166 0.155 0.148 0.133]
  [0.21  0.186 0.166 0.155 0.149 0.134]
  [0.199 0.182 0.167 0.158 0.153 0.141]
  [0.188 0.177 0.167 0.161 0.158 0.149]
  [0.181 0.174 0.167 0.163 0.161 0.154]
  [0.179 0.173 0.167 0.164 0.161 0.156]]

 [[0.233 0.195 0.165 0.149 0.14  0.119]
  [0.229 0.194 0.165 0.15  0.141 0.121]
  [0.208 0.186 0.166 0.156 0.149 0.134]
  [0.192 0.179 0.167 0.16  0.156 0.146]
  [0.183 0.175 0.167 0.163 0.16  0.152]
  [0.182 0.174 0.167 0.163 0.16  0.153]]

 [[0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.   ]
  [0.233 0.195 0.165 0.149 0.14  0.119]
  [0.199 0.182 0.167 0.158 0.153 0.141]
  [0.188 0.177 0.167 0.161 0.158 0.149]
  

In [283]:
i = 6
k = 3
#Wtc = - np.tile(Wt,(k,1)) - np.repeat(goals[:,0],i).reshape((k,i))
#Wtc[Wtc < 0] = 0
#np.repeat(goals[:,0],i).reshape((k,i))
np.tile(Wt,(k,1)) - np.repeat(goals[:,0],i).reshape((k,i))


array([[  9,  10,  20,  40,  60,  65],
       [  0,   1,  11,  31,  51,  56],
       [-11, -10,   0,  20,  40,  45]])

In [295]:
WT1 = np.array([90,95,100,103,105,110])
np.round(calculateTransitionPropabilities(portfolios[2],40,WT1,3,0),3)

array([0.606, 0.226, 0.083, 0.045, 0.03 , 0.011])

```
expected values array([24.794, 29.794, 29.794, 44.794, 95.456, 99.669])
array([[14.794, 15.581, 24.116, 41.942, 95.456, 99.669],
       [24.794, 24.794, 24.794, 24.794, 34.116, 34.116],
       [14.794, 29.794, 29.794, 29.794, 30.581, 39.116],
       [14.794, 15.581, 24.116, 44.794, 44.794, 44.794]])

```

## TODO
- sprawdzić wrażliwość na rożne inputy
- W0 jest większe niż cele

In [260]:
from investment_planner import InvestmentPlanner 

investment_planner = InvestmentPlanner()

goals = np.array([[[0,0]],[[0,0]],[[0,0]],[[0,0]],[[0,0]],[[90,60]],[[0,0]],[[0,0]],[[0,0]],[[0,0]],[[120,100]]])
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
investment_planner.set_params(10, 100, 10, 1, goals, portfolios)

glide_path = investment_planner.glide_paths
wealth_grid = investment_planner.grid
probabilitiesT = investment_planner.probabilitiesT
goal_strategies = investment_planner._goal_strategies

#TODO refactor investment planner - chosen propabilieis to get portfolio_propabilities


ValueError: could not broadcast input array from shape (2,500) into shape (500,500)

In [ ]:
WTt = np.repeat(10,1200)
goals = np.array([[40,10],[49, 15],[60, 30]])

In [448]:
def calculateTransitionPropabilitiesForGoals(Wt, Wt1, infusion, h, goal_costs, portfolioMeasures):
    i0 = len(Wt)
    i1 = len(Wt1)    
    k = len(goal_costs)
    cf = goal_costs - infusion
    mean = portfolioMeasures[0]
    std = portfolioMeasures[1]
    b = ((mean-0.5*std**2)*h)/(std*np.sqrt(h))
    result = np.zeros((k*i0,i1))
    
    Wtc = np.tile(Wt,(k,1)) - cf.reshape((k,1))
    Wtc = Wtc.reshape((k*i0,1))
    Wt1k = np.tile(Wt1, (k*i0,1))
    
    np.divide(Wt1k, Wtc, out=result, where=Wtc>0)
    np.log(result,out=result, where=result>0)
    result = np.where(result > 0, result+ b, 0)
    result = np.where(result > 0, norm.pdf(result), 0)

    result = np.divide(result,np.expand_dims(result.sum(1), axis=1),where=result>0)
    result = result.reshape(k,i0,i1)

    return result
   

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
WT = np.array([49,50,60,80,100,105])
#WT = np.array([100])
WT1 = [90,95,100,103,105,110]

goals = np.array([[40,10],[49, 15],[60, 30]])
#goals = np.array([[49, 15],[60, 30]])
#goals = np.array([[60,0]])
infusion = 5
h = 1

probabilities = calculateTransitionPropabilitiesForGoals(WT, WT1, infusion, h, goals[:,0], [0.1,0.2])

print(np.round(probabilities,3))
probabilities.sum(1)



[[[0.212 0.187 0.166 0.155 0.148 0.132]
  [0.211 0.187 0.166 0.155 0.148 0.133]
  [0.2   0.182 0.167 0.158 0.153 0.14 ]
  [0.189 0.177 0.167 0.161 0.157 0.148]
  [0.181 0.174 0.167 0.163 0.16  0.154]
  [0.18  0.174 0.167 0.163 0.161 0.155]]

 [[0.234 0.195 0.164 0.149 0.139 0.118]
  [0.23  0.194 0.165 0.15  0.141 0.121]
  [0.209 0.186 0.166 0.156 0.149 0.134]
  [0.193 0.179 0.167 0.16  0.156 0.145]
  [0.184 0.176 0.167 0.162 0.159 0.152]
  [0.183 0.175 0.167 0.163 0.16  0.153]]

 [[0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.   ]
  [0.234 0.195 0.164 0.149 0.139 0.118]
  [0.2   0.182 0.167 0.158 0.153 0.14 ]
  [0.189 0.177 0.167 0.161 0.157 0.148]
  [0.186 0.177 0.167 0.162 0.158 0.15 ]]]


array([[1.17265765, 1.08174504, 1.0004663 , 0.95577458, 0.92752447,
        0.86183196],
       [1.23281172, 1.10540205, 0.99677676, 0.9391091 , 0.90339422,
        0.82250615],
       [0.80898429, 0.73189712, 0.66534571, 0.6296638 , 0.60743232,
        0.55667676]])

```
[[0.    0.    0.    0.    0.    0.   ]
 [0.001 0.001 0.001 0.001 0.    0.   ]
 [0.016 0.014 0.012 0.012 0.011 0.01 ]
 [0.091 0.083 0.076 0.072 0.069 0.064]
 [0.177 0.165 0.154 0.148 0.144 0.134]
 [0.197 0.184 0.173 0.166 0.162 0.152]
 [0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]
 [0.046 0.041 0.037 0.034 0.033 0.03 ]
 [0.131 0.121 0.111 0.106 0.103 0.095]
 [0.152 0.141 0.131 0.125 0.122 0.113]]
 ```

In [435]:
arr =  np.array([0.197, 0.184, 0.173, 0.166, 0.162, 0.152])
res = arr/ arr.sum()
np.expand_dims(res, axis=1)


array([[0.19052224],
       [0.17794971],
       [0.16731141],
       [0.16054159],
       [0.15667311],
       [0.14700193]])

In [407]:
log = [1.38778666, 1.44185389, 1.49314718, 1.52270598, 1.54193734, 1.58845736]
log2 =  [3.69037176, 3.74443898, 3.79573227, 3.82529108, 3.84452244, 3.89104245]
norm.pdf(log2)

array([0.00044017, 0.00036002, 0.00029672, 0.00026511, 0.00024626,
       0.00020571])

1: 48.057
2. 64.226
m. 56.305

In [440]:
arr = np.array([1,2,4])
arr1 = np.array([0,1,2])
arr[arr1>0]/ arr1[arr1>0]


array([2., 2.])